In [9]:
import nltk
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense , Activation , Dropout
from keras import utils as np_utils
import keras.preprocessing.text

In [10]:
# load dataset
dataframe = pd.read_excel('AJGT.xlsx', index_col=0) 
dataframe.drop_duplicates(inplace = True)
data= dataframe

print(dataframe.head())

print(type(data['Feed']))

                                                 Feed Sentiment
ID                                                             
1    اربد فيها جامعات اكثر من عمان ... وفيها قد عم...  Positive
2    الحلو انكم بتحكوا على اساس انو الاردن ما فيه ...  Negative
3                             كله رائع بجد ربنا يكرمك  Positive
4                                  لسانك قذر يا قمامه  Negative
5   ​انا داشره وغير متزوجه ولدي علاقات مشبوه واحشش...  Negative
<class 'pandas.core.series.Series'>


In [11]:
train_size = int(len(data) * .7)
test_size =  int(len(data) * .3)

print(int(len(data['Feed'])))
print(train_size)
print(test_size)

1800
1260
540


In [13]:
texts= data['Feed']
tags = data['Sentiment']

train_posts = data['Feed'][:train_size]
train_tags = data['Sentiment'][:train_size]


test_posts = data['Feed'][train_size:]
test_tags =  data['Sentiment'][train_size:]

In [24]:
tokenizer = Tokenizer(num_words=None,lower=False)
tokenizer.fit_on_texts(texts)

# tfidf: the value of the word in the text
# tokanizer: to give every word in the text and convert it to matrix to fit in the model
x_train = tokenizer.texts_to_matrix(train_posts, mode='tfidf')
x_test = tokenizer.texts_to_matrix(test_posts, mode='tfidf')


In [15]:
encoder = LabelEncoder()
encoder.fit(tags)
tagst=encoder.fit_transform(tags)

num_classes = int((len(set(tagst))))
print((len(set(tagst))))

y_train = encoder.fit_transform(train_tags)
y_test = encoder.fit_transform(test_tags)

2


In [26]:
y_train= keras.utils.np_utils.to_categorical(y_train,num_classes)
y_test = keras.utils.np_utils.to_categorical(y_test, num_classes)


num_labels = int(len(y_train.shape))
vocab_size = len(tokenizer.word_index) + 1

max_words=vocab_size
print(max_words)

7075


In [17]:
# Build the model
model = Sequential()
model.add(Dense(1024, input_shape=(max_words,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('sigmoid'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

history = model.fit(x_train, y_train,
                    batch_size=150,
                    epochs=4,
                    verbose=1)
            


model.save('NLP_Model.h1')

Epoch 1/4
9/9 [==============================] - 2s 12ms/step - loss: 0.6212 - accuracy: 0.6540
Epoch 2/4
9/9 [==============================] - 0s 12ms/step - loss: 0.1825 - accuracy: 0.9857
Epoch 3/4
9/9 [==============================] - 0s 12ms/step - loss: 0.0659 - accuracy: 0.9968
Epoch 4/4
9/9 [==============================] - 0s 11ms/step - loss: 0.0279 - accuracy: 0.9992
INFO:tensorflow:Assets written to: NLP_Model.h1/assets


In [18]:
import pickle
# saving
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

# loading
with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

In [19]:
#model = keras.models.load_model('NLP_Model.h1')
Evaluation_valus = model.evaluate(x_test,y_test,verbose=0)
print('accuracy = ',Evaluation_valus[1]*100,'%')

accuracy =  84.44444537162781 %


In [20]:
for x in data["Feed"][:25]:

    tokens = tokenizer.texts_to_matrix([x], mode='tfidf')

    c=model.predict(np.array(tokens))
    predict_x=model.predict(tokens) 
    classes_x=np.argmax(predict_x,axis=1)
    xc = encoder.inverse_transform(classes_x)


    print(c,"= \t",classes_x,"\t",xc)

[[0.03079776 0.9941631 ]] = 	 [1] 	 ['Positive']
[[0.9700677  0.02797674]] = 	 [0] 	 ['Negative']
[[0.08399869 0.96744496]] = 	 [1] 	 ['Positive']
[[0.84861904 0.20516473]] = 	 [0] 	 ['Negative']
[[0.99832505 0.00162815]] = 	 [0] 	 ['Negative']
[[0.01823431 0.99142784]] = 	 [1] 	 ['Positive']
[[0.9869545  0.01593387]] = 	 [0] 	 ['Negative']
[[0.9723547  0.02275095]] = 	 [0] 	 ['Negative']
[[3.3549193e-04 9.9992692e-01]] = 	 [1] 	 ['Positive']
[[0.12324993 0.9211591 ]] = 	 [1] 	 ['Positive']
[[0.9629061  0.04437867]] = 	 [0] 	 ['Negative']
[[0.9647412  0.06022137]] = 	 [0] 	 ['Negative']
[[0.9762832  0.05952402]] = 	 [0] 	 ['Negative']
[[0.90849984 0.17191936]] = 	 [0] 	 ['Negative']
[[0.97406954 0.01847858]] = 	 [0] 	 ['Negative']
[[0.15092535 0.8920725 ]] = 	 [1] 	 ['Positive']
[[0.89923126 0.09626542]] = 	 [0] 	 ['Negative']
[[0.9708374  0.02892057]] = 	 [0] 	 ['Negative']
[[0.862549   0.13951658]] = 	 [0] 	 ['Negative']
[[0.18026152 0.89415383]] = 	 [1] 	 ['Positive']
[[0.11033291 0

In [28]:
#test_input='هذا المحتوى سخيف جدا'
#test_input='هذا المحتوى راقى جدا'
test_input='هذا المحتوى جميل جدا'
input= tokenizer.texts_to_matrix([test_input],mode='tfidf')
c=model.predict(np.array(input))
predict_x=model.predict(input) 
classes_x=np.argmax(predict_x,axis=1)
xc = encoder.inverse_transform(classes_x)

print(c,"= \t",classes_x,"\t",xc)

[[0.27673876 0.8016861 ]] = 	 [1] 	 ['Positive']


In [22]:
#texts= data['Feed']
#tags = data['Sentiment']

#texts=np.array(texts)
#tags=np.array(tags)

#Texts=[]
#for i in range(1800):
#   Sentiment= tags[i]
#   Texts.append((texts[i],Sentiment))

#import random
#random.shuffle(Texts)
#print(Texts)

#all_words = nltk.FreqDist(w for w in texts)
#word_features = list(all_words)[:1800] 
#for i in range(500):
#  print(word_features[0])


#def document_features(document): 
#    document_words = set(document) 
#    features = {}
#    for word in word_features:
#        features['contains({})'.format(word)] = (word in document_words)
#    return features

#document_features(Texts)

#featuresets = [(document_features(d), c) for (d,c) in Texts]
#train_set, test_set = featuresets[:train_size], featuresets[train_size:]

#classifier = nltk.NaiveBayesClassifier.train(train_set)

#print(nltk.classify.accuracy(classifier, test_set)) 